In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:45:41 2020

@author: mahyarfazlyab
"""

# import sys
# sys.path.append("../Python/")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.io import loadmat
import scipy.io
import os
import pickle
import matplotlib.pyplot as plt



def export2matlab(file_name, net, A, B, save_model=False):
    '''
    Export pytorch fully connected network to matlab

    '''

    num_layers = int((len(net) - 1) / 2)
    dim_in = float(net[0].weight.shape[1])
    dim_out = float(net[-1].weight.shape[0])
    hidden_dims = [float(net[2 * i].weight.shape[0]) for i in range(0, num_layers)]

    # network dimensions
    dims = [dim_in] + hidden_dims + [dim_out]

    # get weights
    # weights = np.zeros((num_layers+1,))
    weights = [net[2 * i].weight.detach().numpy().astype(np.float64) for i in range(0, num_layers + 1)]

    # get biases
    # biases = np.zeros((num_layers+1,))
    biases = [net[2 * i].bias.detach().numpy().astype(np.float64).reshape(-1, 1) for i in range(0, num_layers + 1)]

    activation = str(net[1])[0:-2].lower()

    # export network data to matlab
    data = {}
    data['net'] = {'weights': weights, 'biases': biases, 'dims': dims, 'activation': activation, 'name': file_name}
    data['AMatrix'] = A
    data['BMatrix'] = B

    scipy.io.savemat(file_name + '.mat', data)





In [22]:
def main():

    with open('xs.pkl', 'rb') as f:
        Xtrain = pickle.load(f)

    with open('us.pkl', 'rb') as f:
        Ytrain = pickle.load(f)

    # data = loadmat("quadRotorTrainData.mat")
    # Xtrain = data['Xtrain']
    # Ytrain = data['Ytrain']
    # A = data['A']
    # B = data['B']
    if False:
        A = np.zeros((6, 6))
        A[0, 3] = 1
        A[1, 4] = 1
        A[2, 5] = 1

        B = np.zeros((6, 3))
        B[3, 0] =  9.8
        B[4, 1] = -9.8
        B[5, 2] =  1

        A = np.eye(6) + A * 0.1
        B = B * 0.1
    else:
        A = np.array([[1, 1], [0, 1]])
        B = np.array([[0.5], [1]])
    # print(torch.linalg.norm(torch.Tensor(B)), torch.linalg.norm(torch.Tensor(A)))
    # raise
    # print(A)
    # print(B)
    # print(Xtrain.shape)
    # print(Ytrain.shape)
    trainset = torch.utils.data.TensorDataset(torch.Tensor(Xtrain), torch.Tensor(Ytrain))

    net = nn.Sequential(
        nn.Linear(6, 32),
        nn.ReLU(),
        nn.Linear(32, 32),
        nn.ReLU(),
        nn.Linear(32, 3))

    train_batch_size = 128
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size, shuffle=True,
                                              num_workers=2)
    epoch = 500
    net.train()
    losses = []

    def my_loss(output, target):
        gamma = 0.2
        loss = torch.mean((output - target)**2) + gamma * torch.linalg.norm(net[-1].weight) \
                                                    * torch.linalg.norm(net[-3].weight) \
                                                    * torch.linalg.norm(net[-5].weight)
        return loss
        
    # criterion = nn.MSELoss(size_average=None, reduce=None, reduction='mean') + torch.mean(net[-1].weight)
    criterion = my_loss
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    for t in range(epoch):
        for i, (X, Y) in enumerate(trainloader):
            out = net(X)
            loss = criterion(out, Y)

            
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                optimizer.zero_grad()
                for i in [0, 2, 4]:
                    net[i].weight /= torch.maximum(torch.tensor([1]), (torch.linalg.norm(net[i].weight) / 1.))



        losses.append(loss.item())
        if (np.mod(t, 10) == 0):
            print('epoch: ', t, 'MSE loss: ', loss.item())
            print(torch.linalg.norm(net[0].weight) * torch.linalg.norm(net[2].weight) * torch.linalg.norm(net[4].weight))


    export2matlab('quadRotor' , net, A, B, save_model=True)

    return losses

In [23]:

if __name__ == '__main__':
    losses = main()
    plt.plot(losses)


tensor(1.3895) tensor(2.4556)


RuntimeError: No active exception to reraise